In [14]:
#resample data to 1s
dataFrameLatency = dataLatency.resample('1s').ffill()
dataFrame = data.resample('1s').ffill()

In [16]:
np.random.seed(100)

In [17]:
stdScaler = StandardScaler()
minMaxScaler = MinMaxScaler()

In [18]:
stdScalerLatency = StandardScaler()
minMaxScalerLatency = MinMaxScaler()

In [19]:
# Using 3/4 of the data for training
N = (dataFrame.shape[0]*3)//4
D = 1

In [20]:
stdScaler.fit(np.array(dataFrame['value'][:N]).reshape(-1,1))
stdScalerLatency.fit(np.array(dataFrameLatency['value'][:N]).reshape(-1,1))

dataFrame['valueStdScaled'] = dataFrame.apply(lambda row: stdScaler.transform(np.array(row['value']).reshape(-1,1)), 
                                                         axis = 1)
# flatten out the scaled values to match required shape for NN model.fit
dataFrame['valueStdScaled'] = dataFrame.apply(lambda row: row['valueStdScaled'][0][0], axis = 1)

dataFrameLatency['valueStdScaled'] = dataFrameLatency.apply(lambda row: stdScalerLatency.transform(np.array(row['value']).reshape(-1,1)), 
                                                         axis = 1)
# flatten out the scaled values to match required shape for NN model.fit
dataFrameLatency['valueStdScaled'] = dataFrameLatency.apply(lambda row: row['valueStdScaled'][0][0], axis = 1)

In [21]:
minMaxScaler.fit(np.array(dataFrame['value'][:N]).reshape(-1,1))
minMaxScalerLatency.fit(np.array(dataFrameLatency['value'][:N]).reshape(-1,1))

dataFrame['valueMinMaxScaled'] = dataFrame.apply(lambda row: minMaxScaler.transform(np.array(row['value']).reshape(-1,1)), 
                                                            axis = 1)
# flatten out the scaled values to match required shape for NN model.fit
dataFrame['valueMinMaxScaled'] = dataFrame.apply(lambda row: row['valueMinMaxScaled'][0][0], axis = 1)

dataFrameLatency['valueMinMaxScaled'] = dataFrameLatency.apply(lambda row: minMaxScalerLatency.transform(np.array(row['value']).reshape(-1,1)), 
                                                            axis = 1)
# flatten out the scaled values to match required shape for NN model.fit
dataFrameLatency['valueMinMaxScaled'] = dataFrameLatency.apply(lambda row: row['valueMinMaxScaled'][0][0], axis = 1)

In [23]:
dataFrame_train = dataFrame[dataColumnName][:N]
dataFrameLatency_train = dataFrameLatency[dataColumnName][:N]
dataFrame_test = dataFrame[dataColumnName][N:]
dataFrameLatency_test = dataFrameLatency[dataColumnName][N:]

In [28]:
# The data for NN must be in NxTxD format. In this case D=1, since we have a single feature
def formatData(X, Y, series, T, D):
    for t in range(len(series) - T):
        x = series.iloc[t:t+T]
        X.append(x)
        y = series.iloc[t+T]
        Y.append(y)
    
    X = np.array(X).reshape(-1, T, D) # Now the data should be N x T x D
    Y = np.array(Y)
    
    return (X, Y)

In [29]:
#We only need one subset formatted to the size NxTxD for each experiment!

def prepare_dataset(data, T):
    '''
    The config is ther params space dictionary.
    The data is the Series of selected column(s) from the input dataframe, e.g. dataFrame['valueStdScaled'], or dataFrameLatency['valueMinMaxScaled'], etc.
    '''
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    
    (x_train, y_train) = formatData(x_train, y_train, data.iloc[:N], T, D)
    (x_test, y_test) = formatData(x_test, y_test, data.iloc[N:], T, D)   
    
    x_train = np.asarray(x_train).astype(np.float64)
    y_train = np.asarray(y_train).astype(np.float64)
    x_test = np.asarray(x_test).astype(np.float64)
    y_test = np.asarray(y_test).astype(np.float64)    

    return (x_train, y_train, x_test, y_test)